In [1]:
from aloe.program import AloeProgram
text = """
modeh(*,daughter(+person,-person)).
modeb(*,parent(+person,-person)).
modeb(*,parent(-person,+person)).
modeb(*,female(+person)).
modeb(*,female(-person)).
%
determination(daughter/2,parent/2).
determination(daughter/2,female/1).
%    
set(verbose,1).

:- begin_bg.
person(ann).
person(mary).
person(tom).
person(eve).
person(lucy).

parent(ann,mary).
parent(ann,tom).
parent(tom,eve).
parent(tom,lucy).

female(ann).
female(mary).
female(eve).
female(lucy).
:- end_bg.
    

:- begin_in_pos.
daughter(lucy,tom).
daughter(mary,ann).
daughter(eve,tom).
:- end_in_pos.

:- begin_in_neg.
daughter(tom,ann).
daughter(tom,eve).
:- end_in_neg.
"""
ap = AloeProgram.build_from_text(text)
ap.induce(update_B=False, keep_logs=True)
#ap.query('female(X),parent(ann,X),person(Y).')


Examples: 3p/2n
Current example: daughter(lucy,tom).
Bottom_i: daughter(A,B):-female(A),female(C),female(D),female(E),parent(B,A),female(A),female(C),female(D),female(E),parent(C,E),parent(C,B),parent(B,D),parent(B,A).
State: [  0,  1,  3,  2,  2,  0] daughter(A,B).
State: [  1,  1,  3,  0,  1,  1] daughter(A,B):-female(A).
State: [  1,  0,  3,  1,  2,  1] daughter(A,B):-parent(B,D).
State: [  1,  0,  3,  1,  2,  1] daughter(A,B):-parent(B,A).
State: [  2,  0,  3,  0,  1,  1] daughter(A,B):-parent(B,A),female(A).
Clause: daughter(A,B):-female(A).


LogicProgram object
   daughter(A,B):-female(A).

In [2]:
l = ap.learner.logs[0]
l

In [3]:
l.regroup_eventlogs()

In [4]:
l.groups

[{'Examples': [{'pos': [daughter(lucy,tom).,
     daughter(mary,ann).,
     daughter(eve,tom).],
    'neg': [daughter(tom,ann)., daughter(tom,eve).]}],
  'Current example': [daughter(lucy,tom).],
  'Bottom_i': [daughter(A,B):-female(A),female(C),female(D),female(E),parent(B,A),female(A),female(C),female(D),female(E),parent(C,E),parent(C,B),parent(B,D),parent(B,A).],
  'State': [<aloe.hypothesis_metrics.FnMetric._State at 0x1e1086c9790>,
  'Clause': [daughter(A,B):-female(A).]}]

In [2]:
ap.query('not(parent(X,ann)).')

(True, [{}])

In [ ]:
def uses_temporary_options(func, fun_options=None):
    def wrapper(self, *args, **kwargs):
        fun_opt = {key:kwargs[key] for key in fun_options if key in kwargs}
        temp_options = {key:value for key,value in kwargs.items() if key not in fun_opt}
        self.add_temporary_options(**temp_options)
        out = func(*args, **fun_opt)
        self.rem_temporary_options()
        return out
    return wrapper


In [38]:
import inspect
def fun(x, c=1, *t, a=1, b=3):
    pass

def decorum(fun):
    i = inspect.getfullargspec(fun)
    def wrap(*args, **kwargs):
        fun_args = i.args[len(args):] + i.kwonlyargs
        fun_options = {key:kwargs[key] for key in fun_args if key in kwargs}
        temp_options = {key:value for key,value in kwargs.items() if key not in fun_options}
        
        out = fun(*args, **fun_options)
        
        return out
    
i = inspect.getfullargspec(fun)
print(i)
i.args[-len(i.defaults):] + kwonlyargs

FullArgSpec(args=['x', 'c'], varargs='t', varkw=None, defaults=(1,), kwonlyargs=['a', 'b'], kwonlydefaults={'a': 1, 'b': 3}, annotations={})


['c']

In [42]:
def fun(x, c=1, *t, a=1, b=3):
    pass
fun(1,c=4)

In [14]:
def foo(arg1, arg2, arg3, *args, kwarg1="FOO", kwarg2="BAR", kwarg3="BAZ"):
    pass

print(foo.__kwdefaults__)

{'kwarg1': 'FOO', 'kwarg2': 'BAR', 'kwarg3': 'BAZ'}


In [3]:
from aloe.parser import AloeParser
parser = AloeParser()
parser.parse_query('not(parent(X,tom)).')

not(parent(X,tom))

In [3]:
""" Display of results for ProgolLearner with the FnMetric """
from aloe.program import AloeProgram
from collections import OrderedDict

ap = AloeProgram.build_from_file("aloe/Examples/family.pl")
ap.induce(update_B=False, keep_logs=True, verbose=0)

log = ap.learner.logs[0]
log.regroup_eventlogs()

In [4]:
groups = OrderedDict((group['Current example'][0],i) for i, group in enumerate(log.groups))

In [6]:
"Examples, Background, Modes"

import ipywidgets as widgets

attribute_description = {
    'p':'Number of positive examples covered',
    'n':'Number of negative examples covered',
    'f':'Fitness value\nf=p-(n+c+h)',
    'g':'Upper bound on fitness value\ng=p-(c+h)',
    'h':'Optimistic estimate of atoms needed',
    'c':'Number of atoms in the body',
}

E = widgets.Dropdown(
    options=list(groups),
    description='Example:',
    disabled=False,
)

# Create table for all states
#----------------------------

def explain_p(attr, s, grid):
    examples = s.E_cov
    w = widgets.Dropdown(
        options=s.E_cov['pos'],
        description='Positive examples:',
    )        
    def explanation_button(button):
        grid[0, :5] = w
        grid[0,5] = widgets.Label('p=%d/%d' % (len(s.E_cov['pos']), len(s.hm.E['pos'])),
                                   placeholder='start, stop[, pack] [last, pack]')
    return explanation_button

def explain_n(attr, s, grid):
    examples = s.E_cov
    def explanation_button(button):
        w = widgets.Dropdown(
            options=s.E_cov['neg'],
            description='Negative examples:',
        )        
        grid[0, :5] = w
        grid[0,5] = widgets.Label('n=%d/%d' % (len(s.E_cov['neg']), len(s.hm.E['neg'])))
    return explanation_button


def button(grid, attr, s=None):        
    if s is None:
        descr = attr
        b_style = 'success'
    else:
        descr = str(getattr(s, attr))
        b_style = 'warning'

    if attr=='clause':
        layout = widgets.Layout(display="flex", justify_content="flex-start", height='auto', width='auto')
    else:
        layout = widgets.Layout(height='auto', width='auto')

    button = widgets.Button(description=descr, button_style=b_style, layout=layout)
    
    if attr=='p' and s is not None:
        button.on_click(explain_p(attr, s, grid))
    return button

def details_all_states(e):
    """ Input: e, a clause learned """
    
    log_e = log.groups[groups[e]]
    
    states = log_e['State']
    n_states = len(states)
    
    # Fn measure part    
    state_attributes = ['f', 'g', 'p', 'n', 'c', 'h']
    n_attributes = len(state_attributes)
    
    grid = widgets.GridspecLayout(n_states+2, n_attributes)
    
    for j, attr in enumerate(state_attributes):
        b = button(grid, attr)
        if attr in attribute_description:
            b.tooltip = attribute_description[attr]
        grid[1, j] = b 
    for i, s in enumerate(states):
        for j, attr in enumerate(state_attributes):
            grid[i+2, j] = button(grid, attr, s)
            
    # Clause part
    attr = 'clause'
    clause_widget = widgets.GridspecLayout(n_states+2, 1)
    clause_widget[1, 0] = button(grid, attr)
    for i, s in enumerate(states):
        clause_widget[i+2, 0] = button(grid, attr, s)
          
    # States_widget
    states_widget = widgets.AppLayout(left_sidebar=clause_widget, center=grid, pane_widths=(.6,.4,0))
            
    return states_widget    

tables = {e:details_all_states(e) for e in list(groups)}

def fun(e):
    return tables[e]

widgets.interact(fun, e=E)

interactive(children=(Dropdown(description='Example:', options=(parent_of(ada,sylvia)., parent_of(george,oscar…

<function __main__.fun(e)>

In [7]:
groups

OrderedDict([(parent_of(ada,sylvia)., 0),
             (parent_of(george,oscar)., 1),
             (grandfather_of(george,bernard)., 2),
             (grandparent_of(ada,andrew)., 3)])

In [18]:
grid = tables['parent_of(ada,sylvia).'].center
grid[:3,0] = button('clause')


In [17]:
help(widgets.Button.tooltip)

Help on Unicode in module traitlets.traitlets:




In [26]:
l = widgets.Label('p=%d/%d' % (1, 2), 
                  description_tooltip='start, stop[, pack] [last, pack]')
l.tooltip = "dfqnlmk"
l

Label(value='p=1/2', description_tooltip='start, stop[, pack] [last, pack]')

In [33]:
b = widgets.Button(description='qdlpfjqi', button_style='warning')
b.tooltip = 'qmf'
b

Button(button_style='warning', description='qdlpfjqi', style=ButtonStyle(), tooltip='qmf')

In [19]:

help(grid)

Help on GridspecLayout in module ipywidgets.widgets.widget_templates object:

class GridspecLayout(ipywidgets.widgets.widget_box.GridBox, LayoutProperties)
 |  GridspecLayout(n_rows=None, n_columns=None, **kwargs)
 |  
 |  Define a N by M grid layout
 |  
 |  Parameters
 |  ----------
 |  
 |  n_rows : int
 |      number of rows in the grid
 |  
 |  n_columns : int
 |      number of columns in the grid
 |  
 |  grid_gap : str
 |      CSS attribute used to set the gap between the grid cells
 |  
 |  justify_content : str, in ['flex-start', 'flex-end', 'center', 'space-between', 'space-around']
 |      CSS attribute used to align widgets vertically
 |  
 |  align_items : str, in ['top', 'bottom', 'center', 'flex-start', 'flex-end', 'baseline', 'stretch']
 |      CSS attribute used to align widgets horizontally
 |  
 |  width : str
 |  height : str
 |      width and height
 |  
 |  Examples
 |  --------
 |  
 |  >>> from ipywidgets import GridspecLayout, Button, Layout
 |  >>> layout = Gr

In [62]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

    def green(text): return bcolors.OKGREEN + text + bcolors.ENDC
    def red(text):   return bcolors.FAIL    + text + bcolors.ENDC


In [65]:
print(bcolors.green('qmldj'))

qmldj


In [59]:
def strike(text):
    return ''.join([u'\u0336{}'.format(c) for c in text])
print(strike('mqsd'))

̶m̶q̶s̶d


In [9]:
button = widgets.Button(description='Click here')
slider = widgets.IntSlider()
stacked = widgets.Stacked([button, slider])
stacked  # will show only the button

AttributeError: module 'ipywidgets' has no attribute 'Stacked'

In [ ]:
widgets.

In [1]:
from aloe.program import AloeProgram
from aloe.options import SystemParameters
import time
ap2 = AloeProgram.build_from_file("aloe/Examples/family.pl")
ap2.set('verbose',1)
t0 = time.time()
SystemParameters.generic_name_for_variable = True
out = ap2.induce()
t1 = time.time()
t1-t0

Examples: 66p/100n
Current example: parent(ada,sylvia).
Bottom_i: parent(A,B):-mother(A,B),mother(B,C),mother(B,D),mother(B,E),mother(B,F).
State: [  0,  1, 22, 48, 21,-27] parent(A,B).
State: [  1,  0, 10,  0,  9,  9] parent(A,B):-mother(A,B).
Clause: parent(A,B):-mother(A,B).

Examples: 56p/100n
Current example: parent(george,oscar).
Bottom_i: parent(A,B):-father(A,B),father(B,C),father(B,D),father(B,E),father(B,F),father(B,G).
State: [  0,  1, 12, 48, 11,-37] parent(A,B).
State: [  1,  0, 12,  0, 11, 11] parent(A,B):-father(A,B).
Clause: parent(A,B):-father(A,B).

Examples: 44p/100n
Current example: grandfather(george,bernard).
Bottom_i: grandfather(A,B):-father(A,C),parent(A,C),father(C,D),father(C,E),father(C,F),father(C,B),father(C,G),parent(C,D),parent(C,E),parent(C,F),parent(C,B),parent(C,G).
State: [  0,  2, 18, 30, 16,-14] grandfather(A,B).
State: [  1,  1, 18, 12, 16,  4] grandfather(A,B):-father(A,C).
State: [  2,  0, 18,  0, 16, 16] grandfather(A,B):-father(A,C),parent(C,B

1.376328945159912

In [1]:
from aloe.program import AloeProgram
from collections import OrderedDict

ap = AloeProgram.build_from_file("aloe/Examples/family.pl")
ap.induce(update_B=False, keep_logs=True, verbose=0)
ap.display_logs()

interactive(children=(Dropdown(description='Example:', options=(parent(ada,sylvia)., parent(george,oscar)., gr…

In [5]:
import ipywidgets
h = ipywidgets.HTML(value = f"<b><font color='red'>{1}</b>")

In [6]:
m = ipywidgets.Button(button_style = 'warning')
m.button_style = 'success'
display(m)

Button(button_style='success', style=ButtonStyle())

In [3]:
with open("aloe/Examples/family.pl", 'r') as file:
    text = file.read()
    
with open("aloe/Examples/family.pl", 'w') as file:
    file.write(text.replace('_of',''))

In [2]:
t = '_of dkmq'
t.replace('_of','')

' dkmq'

In [17]:
m.style.button_color = None

In [5]:
c_base = int("1F534",base=16)
c_base
'{} {}'.format(chr(c_base), 'mkdq')

'🔴 mkdq'

In [24]:
d = ipywidgets.Select(options = ["\033[32m Bright Green", "\033[37m Bright Red"])
display(d)

Select(options=('\\033 Bright Green', '\\033 Bright Red  \\n'), value='\\033 Bright Green')

In [6]:
help(ipywidgets.Dropdown)

Help on class Dropdown in module ipywidgets.widgets.widget_selection:

class Dropdown(_Selection)
 |  Dropdown(*args, **kwargs)
 |  
 |  Allows you to select a single item from a dropdown.
 |  
 |  Parameters
 |  ----------
 |  options: list
 |      The options for the dropdown. This can either be a list of values, e.g.
 |      ``['Galileo', 'Brahe', 'Hubble']`` or ``[0, 1, 2]``, or a list of
 |      (label, value) pairs, e.g.
 |      ``[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)]``.
 |  
 |  index: int
 |      The index of the current selection.
 |  
 |  value: any
 |      The value of the current selection. When programmatically setting the
 |      value, a reverse lookup is performed among the options to check that
 |      the value is valid. The reverse lookup uses the equality operator by
 |      default, but another predicate may be provided via the ``equals``
 |      keyword argument. For example, when dealing with numpy arrays, one may
 |      set ``equals=np.array_equal``.
 |

In [3]:
ap2.verify('grandparent_of(A,B):-grandfather_of(A,B).')

not(grandfather_of(A,B),not(grandparent_of(A,B)))


(True, [{}])

In [4]:
ap2.verify('grandfather_of(A,B):-grandparent_of(A,B).')

not(grandparent_of(A,B),not(grandfather_of(A,B)))


(False, [])

In [7]:
from aloe.knowledge import MultipleKnowledge, LogicProgram
ap = AloeProgram.build_from_text(text)
ap.knowledge = MultipleKnowledge(LogicProgram(options=ap.options), ap.knowledge, options=ap.options)
print(ap.query('female(mary).'))



h 0
Atom female(mary)
Candidates {female(mary).}
Match {female(mary).}
Clause female(mary).
Substitution {}
Atoms []
[{}]


In [2]:
female(X), not(parent(X,Y)), female(Y)

TypeError: object of type 'NoneType' has no len()

In [1]:
from aloe.clause import Clause, Predicate, Function, Constant, Variable, Operator
def print_class_in_depth(element):
    print(element.__class__, element)
    if isinstance(element, Operator):
        for t in element:
            print_class_in_depth(t)

atom = Predicate('female', [Constant('mary')])
print_class_in_depth(atom)            

<class 'aloe.clause.Predicate'> female(mary)
<class 'aloe.clause.Constant'> mary


In [10]:
def fun(a, b, **options):
    print(*options)
    
l = [1,2]
fun(*l, h=30, t=50)


h t


In [3]:
import aloe.program

t = """
mortel(X):-man(X).
mortel(X):-animal(X).

man(socrates).
animal(lion).
"""

ap = aloe.program.AloeProgram.build_from_background(t)

q = "mortel(X),not(man(X))."
ap.query(q)


(True, [{X: lion}])

In [5]:
a = {1,2,3}
b = {4,5}
c = {3,4}
a-c

{1, 2}


<div class="dropdown">
  <button class="dropbtn">Dropdown</button>
  <div class="dropdown-content">
    <a href="#">Link 1</a>
    <a href="#">Link 2</a>
    <a href="#">Link 3</a>
  </div>
</div>

In [6]:
""" %d qdsjfmqjq""" % 3

' 3 qdsjfmqjq'

In [4]:
set.union({1,2},{3})

{1, 2, 3}

In [1]:
def fun():
    print(__name__)
    
fun()

__main__


In [1]:
from aloe.options import Options
class A:
    def __init__(self, options=None):
        self._options = options if options else Options()
        self._temp_options = dict()
        
    def __getattr__(self, attr):
        if attr=='options':
            if attr in self._temp_options:
                return self._temp_options[attr]
            else:
                return self._options, attr)
        else: return super().__getattr__(self)


In [3]:
a = A()
a.options.a

AttributeError: 'Options' object has no attribute 'options'

In [6]:
a.__dir__()

['_options',
 '_temp_options',
 '__module__',
 '__init__',
 '__getattr__',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [2]:
class A:
    def __init__(self, s: str):
        self.s = s
a = A(1)

In [4]:
a = {1}
isinstance(a , list)



False

In [8]:
help(iterable)

Help on class tuple in module builtins:

class tuple(object)
 |  tuple(iterable=(), /)
 |  
 |  Built-in immutable sequence.
 |  
 |  If no argument is given, the constructor returns an empty tuple.
 |  If iterable is specified the tuple is initialized from iterable's items.
 |  
 |  If the argument is a tuple, the return value is the same object.
 |  
 |  Built-in subclasses:
 |      asyncgen_hooks
 |      UnraisableHookArgs
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __

In [12]:
raise NotImplementedError()

NotImplementedError: 

In [2]:
'a b \n c'.split()

['a', 'b', 'c']

In [3]:
a = #qmdfk
a

SyntaxError: invalid syntax (Temp/ipykernel_2252/2001087664.py, line 1)

In [2]:
any(True for _ in range(3))

True

In [3]:
isinstance((3,1),(int,int))

False

In [8]:
class SubstitutionError(Exception):
    def __init__(self, key, item):
        message = 'Substitution {%s/%s} is not valid' % (str(key), str(item))
        super().__init__(message)
        
raise SubstitutionError(3,4)

SubstitutionError: Substitution {3/4} is not valid

In [14]:
import os

def switch_Function_CompoundTerm(fname):
    f = open(fname, 'r')
    t = f.read()
    f.close()

    t = t.replace("Function", "FFFFF")
    t = t.replace("CompoundTerm", "Function")
    t = t.replace("FFFFF", "CompoundTerm")

    f = open(fname, 'w')
    f.write(t)
    f.close()

    
    
"""
directory = "aloe"
# iterate over files in
# that directory
for filename in os.listdir(directory):
    fname = os.path.join(directory, filename)
    if os.path.isfile(fname) and fname[-3:]==".py":
        switch_Function_CompoundTerm(fname)

"""

'\ndirectory = "aloe"\n# iterate over files in\n# that directory\nfor filename in os.listdir(directory):\n    fname = os.path.join(directory, filename)\n    if os.path.isfile(fname) and fname[-3:]==".py":\n        switch_Function_CompoundTerm(fname)\n\n'

In [3]:
"""
import os
directory = "aloe"
# iterate over files in
# that directory
for filename in os.listdir(directory):
    fname = os.path.join(directory, filename)
    if os.path.isfile(fname) and fname[-3:]==".py":
        f = open(fname, 'r')
        t = f.read()
        f.close()

        t = t.replace("ModeHandler", "ModeCollection")

        f = open(fname, 'w')
        f.write(t)
        f.close()        
"""

In [2]:
from itertools import chain

In [3]:
'a'.capitalize

Help on class chain in module itertools:

class chain(builtins.object)
 |  chain(*iterables) --> chain object
 |  
 |  Return a chain object whose .__next__() method returns elements from the
 |  first iterable until it is exhausted, then elements from the next
 |  iterable, until all of the iterables are exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __class_getitem__(...) from builtins.type
 |      See PEP 585
 |  
 |  from_iterable(iterable, /) from builtins.type
 |      Alternative chain() constructor taking a single iterable argument tha

In [5]:
class A:
    def __init__(self):
        self.l = [1,2,3]
        
    def __iter__(self):
        return iter(self.l)
    
a = A()
list(a)

[1, 2, 3]

In [19]:
s = 'mqkdm'
s.capitalize()

'Mqkdm'

In [11]:
def fun(x):
    if x:
        yield x
        
g = fun(0)


SyntaxError: unexpected EOF while parsing (Temp/ipykernel_10752/2609936294.py, line 7)

In [16]:
ord('A')

65

In [17]:
for a in range('A','Z'):
    print(a)

TypeError: 'str' object cannot be interpreted as an integer

In [2]:
class A:
    a = 5
    
A.a = 6
a = A()
a.a

6

In [6]:
s = """
ada parent_of andrew.
ada parent_of john.
ada parent_of robert.
ada parent_of stephen.
edith parent_of andrew.
edith parent_of john.
edith parent_of robert.
edith parent_of stephen.
george parent_of bernard.
george parent_of cathleen.
george parent_of daphne.
george parent_of fred.
george parent_of louis.
oscar parent_of andrew.
oscar parent_of john.
oscar parent_of robert.
oscar parent_of stephen.
paul parent_of bernard.
paul parent_of cathleen.
paul parent_of daphne.
paul parent_of fred.
paul parent_of louis.
william parent_of andrew.
william parent_of john.
william parent_of robert.
william parent_of stephen.

sylvia parent_of ada.
bernard parent_of edith.
cathleen parent_of edith.
daphne parent_of edith.
fred parent_of edith.
louis parent_of edith.
oscar parent_of george.
andrew parent_of louis.
john parent_of louis.
robert parent_of louis.
stephen parent_of louis.
bernard parent_of oscar.
cathleen parent_of oscar.
daphne parent_of oscar.
fred parent_of oscar.
louis parent_of oscar.
edith parent_of paul.
andrew parent_of sylvia.
john parent_of sylvia.
robert parent_of sylvia.
stephen parent_of sylvia.
sylvia parent_of william.
ada grandparent_of sylvia.

edith grandparent_of bernard.
edith grandparent_of cathleen.
edith grandparent_of daphne.
edith grandparent_of fred.
edith grandparent_of louis.
george grandparent_of oscar.
louis grandparent_of andrew.
louis grandparent_of john.
louis grandparent_of robert.
louis grandparent_of stephen.
oscar grandparent_of bernard.
oscar grandparent_of cathleen.
oscar grandparent_of daphne.
oscar grandparent_of fred.
oscar grandparent_of louis.
paul grandparent_of edith.
sylvia grandparent_of andrew.
sylvia grandparent_of john.
sylvia grandparent_of robert.
sylvia grandparent_of stephen.
william grandparent_of sylvia.

ada grandfather_of sylvia.
edith grandfather_of bernard.
edith grandfather_of cathleen.
edith grandfather_of daphne.
edith grandfather_of fred.
edith grandfather_of louis.
george grandfather_of oscar.
louis grandfather_of andrew.
louis grandfather_of john.
louis grandfather_of robert.
louis grandfather_of stephen.
oscar grandfather_of bernard.
oscar grandfather_of cathleen.
oscar grandfather_of daphne.
oscar grandfather_of fred.
oscar grandfather_of louis.
paul grandfather_of edith.
sylvia grandfather_of andrew.
sylvia grandfather_of john.
sylvia grandfather_of robert.
sylvia grandfather_of stephen.
william grandfather_of sylvia.

ada grandfather_of andrew.
ada grandfather_of john.
ada grandfather_of robert.
ada grandfather_of stephen.
edith grandfather_of andrew.
edith grandfather_of john.
edith grandfather_of robert.
edith grandfather_of stephen.
"""
s = s.replace(".","")
out = ''
for line in s.split("\n"):
    if line: 
        a, p, b, *garbage = line.split(' ')
        out += '%s(%s,%s).' % (p, a, b)
    out += '\n'

print(out)


parent_of(ada,andrew).
parent_of(ada,john).
parent_of(ada,robert).
parent_of(ada,stephen).
parent_of(edith,andrew).
parent_of(edith,john).
parent_of(edith,robert).
parent_of(edith,stephen).
parent_of(george,bernard).
parent_of(george,cathleen).
parent_of(george,daphne).
parent_of(george,fred).
parent_of(george,louis).
parent_of(oscar,andrew).
parent_of(oscar,john).
parent_of(oscar,robert).
parent_of(oscar,stephen).
parent_of(paul,bernard).
parent_of(paul,cathleen).
parent_of(paul,daphne).
parent_of(paul,fred).
parent_of(paul,louis).
parent_of(william,andrew).
parent_of(william,john).
parent_of(william,robert).
parent_of(william,stephen).

parent_of(sylvia,ada).
parent_of(bernard,edith).
parent_of(cathleen,edith).
parent_of(daphne,edith).
parent_of(fred,edith).
parent_of(louis,edith).
parent_of(oscar,george).
parent_of(andrew,louis).
parent_of(john,louis).
parent_of(robert,louis).
parent_of(stephen,louis).
parent_of(bernard,oscar).
parent_of(cathleen,oscar).
parent_of(daphne,oscar).
pa

In [3]:
help(tuple)

Help on class tuple in module builtins:

class tuple(object)
 |  tuple(iterable=(), /)
 |  
 |  Built-in immutable sequence.
 |  
 |  If no argument is given, the constructor returns an empty tuple.
 |  If iterable is specified the tuple is initialized from iterable's items.
 |  
 |  If the argument is a tuple, the return value is the same object.
 |  
 |  Built-in subclasses:
 |      asyncgen_hooks
 |      UnraisableHookArgs
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __

In [5]:
from aloe.utils import generate_variable_names
t0 = time.time()
generate_variable_names()
time.time()-t0

0.0

In [4]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self))

In [1]:
class A:
    def __init__(self):
        self.a = 3
        self.b = 4
    def fun(self,c):
        self.c = c
        
a = A()
t = a.fun
t(5)
print(a.c)
t(6)
print(a.c)

5
6


In [13]:
def fun():
    a = {2:3}
    print(dir())
    print(locals())

In [14]:
b = 5
fun()

['a']
{'a': {2: 3}}


In [3]:
class Inf(int):
    def __init__(self)

a = A(1)
a

1

In [10]:
class A(list):
    pass

b = A((1,))
a = A((1,)) + A((3,4)) 
a.__class__

list

In [2]:
class A:
    def __init__(self):
        super(A, self).__init__()
        self.a = 3
        self.b = 5
        print('A')
        
class B:
    def __init__(self):
        super(B, self).__init__()
        self.b = 4
        self.c = 6
        print('B')
        
class C(A,B):
    def __init__(self):
        super(C, self).__init__()
        print('C')
c = C()
c.a, c.b, c.c

B
A
C


(3, 5, 6)

In [3]:
help(super)

Help on class super in module builtins:

class super(object)
 |  super() -> same as super(__class__, <first argument>)
 |  super(type) -> unbound super object
 |  super(type, obj) -> bound super object; requires isinstance(obj, type)
 |  super(type, type2) -> bound super object; requires issubclass(type2, type)
 |  Typical use to call a cooperative superclass method:
 |  class C(B):
 |      def meth(self, arg):
 |          super().meth(arg)
 |  This works for class methods too:
 |  class C(B):
 |      @classmethod
 |      def cmeth(cls, arg):
 |          super().cmeth(arg)
 |  
 |  Methods defined here:
 |  
 |  __get__(self, instance, owner, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  --------------------------

In [6]:
def fun():
    yield 1
    yield 2
    print('Finish')

g = fun()
print(next(g, None))
print(next(g, None))
print(4)
print(next(g, None))


1
2
4
Finish
None
